In [1]:
import pandas as pd
import re

In [2]:
stu_df = pd.read_csv('qgdata/student.csv')
# dict(stu_df.department.value_counts())

In [3]:
import numpy as np
import scipy.special
from bokeh.models import ColumnDataSource, FactorRange, LabelSet
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show

output_file('report.html', mode='inline')
# output_notebook()

# 人数统计图
groups = ['后台组', '数据挖掘组', '前端组', '嵌入式组', '移动组', '图形组', '设计组']
mgroups = ['bg', 'dm', 'web', 'eb', 'app', 'gf', 'ds']
years = ['2017', '2018', '2019']

data = {'groups' : groups,
        '2017'   : [88, 52, 65, 33, 58, 43, 14],
        '2018'   : [118, 63, 51, 44, 50, 29, 22],
        '2019'   : [52, 47, 46, 27, 21, 16, 10]}

x = [ (group, year) for group in groups for year in years ]
counts = sum(zip(data['2017'], data['2018'], data['2019']), ())

source = ColumnDataSource(data=dict(x=x, counts=counts))

p_num = figure(x_range=FactorRange(*x), plot_height=400, plot_width=800, title="历年各小组报名人数",
           toolbar_location=None, tools="hover")

p_num.vbar(x='x', top='counts', width=0.9, source=source)

p_num.y_range.start = 0
p_num.y_range.end = 140
p_num.x_range.range_padding = 0.1
p_num.xaxis.major_label_orientation = 1
p_num.xgrid.grid_line_color = None
p_num.align = 'center'

labels = LabelSet(x='x', y='counts', text='counts', level='glyph',
            x_offset=-9, y_offset=0, source=source, render_mode='canvas')


p_num.add_layout(labels)

In [4]:
# 数据处理
from math import pi
import pandas as pd
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum
from bokeh.layouts import row


def pie(data, title):
    p = figure(plot_height=350, plot_width=450, title=title, toolbar_location=None,
               tools="hover", tooltips="@country: @value", x_range=(-0.5, 1.0))

    p.wedge(x=0, y=1, radius=0.4,
            start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
            line_color="white", fill_color='color', legend_field='country', source=data)

    p.axis.axis_label=None
    p.axis.visible=False
    p.grid.grid_line_color = None

    return p


def make_plot(title, hist, edges, x, pdf):
    p = figure(title=title, tools='', background_fill_color="#fafafa")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="navy", line_color="white", alpha=0.5)
    p.line(x, pdf, line_color="#ff8888", line_width=4, alpha=0.7, legend_label='正态分布')

    p.y_range.start = 0
    p.xaxis.axis_label = '绩点'
    p.yaxis.axis_label = '概率密度'
    p.grid.grid_line_color="white"
    return p



department = stu_df.department.replace({r'\(?（?[一二三四五六七八九]?\d{0,2}\)?）?班? ?$': '',
                                        r'^(20)?18级?': '',
                                        r'(20)?18级?$': '',
                                        r'信息工程学院电子信息类|信工电子信息类|信工电子信息|信工|信息工程学院': '信息工程学院',
                                        r'信安|信息安全': '计算机学院',
                                        r'光电信息$|光电信息科学与工程$': '其他学院',
                                        r'(数据科学与)?大数据?(技术)?$': '自动化学院',
                                        '数据科学与大数据技术': '自动化学院',
                                        '自动化':'自动化学院',
                                       '机械$|机械类': '自动化学院',
                                      '物联网$|物联网工程': '自动化学院',
                                      '电子信息$|电子信息类': '信息工程学院',
                                      '计科|计算机$|计算机科学与技术': '计算机学院',
                                      '(软)?件$|软$|软件工程专业$|软工$|软件工程': '计算机学院',
                                      '土木$|土木类$': '其他学院',
                                      '能动|能源与动力工程': '其他学院',
                                      '网工$|网络工程': '计算机学院',
                                      '自动化卓越|自动化$': '自动化学院',
                                       '电子科学与技术':'物理与光电工程学院',
                                        '光电信息科学与工程':'物理与光电工程学院',
                                      '专业':'',}, regex=True)
dlist = ['计算机学院','信息工程学院','自动化学院','物理与光电工程学院']
for i in range(len(department)):
    if department[i] not in dlist:
        department[i] = '其他学院'
        
def department_distribution(src):
    department = src.department.replace({r'\(?（?[一二三四五六七八九]?\d{0,2}\)?）?班? ?$': '',
                                        r'^(20)?18级?': '',
                                        r'(20)?18级?$': '',
                                        r'信息工程学院电子信息类|信工电子信息类|信工电子信息|信工|信息工程学院': '信息工程学院',
                                        r'信安|信息安全': '计算机学院',
                                        r'光电信息$|光电信息科学与工程$': '其他学院',
                                        r'(数据科学与)?大数据?(技术)?$': '自动化学院',
                                        '数据科学与大数据技术': '自动化学院',
                                        '自动化':'自动化学院',
                                       '机械$|机械类': '自动化学院',
                                      '物联网$|物联网工程': '自动化学院',
                                      '电子信息$|电子信息类': '信息工程学院',
                                      '计科|计算机$|计算机科学与技术': '计算机学院',
                                      '(软)?件$|软$|软件工程专业$|软工$|软件工程': '计算机学院',
                                      '土木$|土木类$': '其他学院',
                                      '能动|能源与动力工程': '其他学院',
                                      '网工$|网络工程': '计算机学院',
                                      '自动化卓越|自动化$': '自动化学院',
                                       '电子科学与技术':'物理与光电工程学院',
                                        '光电信息科学与工程':'物理与光电工程学院',
                                      '专业':'',}, regex=True)
    dlist = ['计算机学院','信息工程学院','自动化学院','物理与光电工程学院']
    for i, v in department.items():
        if department[i] not in dlist:
            department[i] = '其他学院'
    return dict(department.value_counts())

def subject_distribution(src):
    subject = src.department.replace({r'\(?（?[一二三四五六七八九]?\d{0,2}\)?）?班? ?$': '',
                                            r'^(20)?18级?': '',
                                            r'(20)?18级?$': '',
                                            r'信息工程学院电子信息类|信工电子信息类|信工电子信息|信工': '信息工程学院',
                                            r'信安': '信息安全',
                                            r'光电信息$': '光电信息科学与工程',
                                            r'(数据科学与)?大数据$': '数据科学与大数据技术',
                                           '机械$': '机械类',
                                          '物联网$': '物联网工程',
                                          '电子信息$': '电子信息类',
                                          '计科': '计算机科学与技术',
                                          '(软)?件$|软$|软件工程专业$|软工$': '软件工程',
                                          '土木$': '土木类',
                                          '电子科学与技术三八': '电子科学与技术',
                                          '能动': '能源与动力工程',
                                          '网工$': '网络工程',
                                          '自动化卓越': '自动化',
                                          '专业':''}, regex=True)
    subject.replace({r'^(?!(信息安全)|(数据科学与大数据技术)|(电子信息类)|(网络工程)|(计算机科学与技术)|(软件工程)|(电子科学与技术)).*$': '其他专业'},regex=True, inplace=True)
    return dict(subject.value_counts())


department_dict = dict()
subject_dict = dict()
sex_dict = dict()
for group in groups:
    department_dict[group] = department_distribution(stu_df[stu_df['group'] == group])
    subject_dict[group] = subject_distribution(stu_df[stu_df['group'] == group])
    sex_dict[group] = dict(stu_df[stu_df['group'] == group].sex.value_counts())

department_dict['总体'] = department_distribution(stu_df)
subject_dict['总体'] = subject_distribution(stu_df)
sex_dict['总体'] = dict(stu_df.sex.value_counts())


src_dict = dict()

groups.append('总体')
mgroups.append('tot')
for group, mgroup in zip(groups, mgroups):
    for dic, name in zip([department_dict, subject_dict, sex_dict], ['dep_', 'sbj_', 'sex_']):
        x = dic[group]
        data = pd.Series(x).reset_index(name='value').rename(columns={'index':'country'})
        data['angle'] = data['value']/data['value'].sum() * 2*pi
        if len(x) == 2:
            data['color'] = ['black', 'red']
        else:
            data['color'] = Category20c[len(x)]
        src_dict[name + mgroup] = ColumnDataSource(data)
        if mgroup == 'tot':
            src_dict[name + 'df'] = ColumnDataSource(data)    

# di = dict(stu_df.department.replace({' ?': '',
#                                 '一': '1', '二': '2', '三': '3', '四': '4', 
#                                 '五': '5', '六': '6', '七': '7', '八': '8', 
#                                 '九': '9',
#                                '计科': '计算机科学与技术'}, regex=True).replace(
#     {r'^(?!(软件工程3班)|(软件工程4班)|(计算机科学与技术3班)|(计算机科学与技术4班)).*$': '其他班级'},
#     regex=True).value_counts())
# di['其他班级'] -= 6
# di['未填写计科软工班级'] = 6

# data = pd.Series(di).reset_index(name='value').rename(columns={'index':'country'})
# data['angle'] = data['value']/data['value'].sum() * 2*pi
# data['color'] = Category20c[len(di)]
# src_dict['qg'] = ColumnDataSource(data)
# p_qg = pie(src_dict['qg'], '谢老师班级分布')

In [5]:
from bokeh.layouts import column, layout
from bokeh.models import Select, CustomJS


# 3个饼图

p_dpm = pie(src_dict['dep_df'], '学院分布')
p_spj = pie(src_dict['sbj_df'], '专业分布')
p_sex = pie(src_dict['sex_df'], '性别分布')

select = Select(title="选择饼图统计范围:", value="总体", 
                options=['总体', '后台组', '数据挖掘组', '前端组', '嵌入式组', '移动组', '图形组', '设计组'])


# 设置回调函数
callback = CustomJS(args=src_dict, code="""
     if(cb_obj.value === "总体"){ 
             dep_df.data = dep_tot.data;
             sbj_df.data = sbj_tot.data;
             sex_df.data = sex_tot.data;
     }else if(cb_obj.value === "后台组"){
             dep_df.data = dep_bg.data;
             sbj_df.data = sbj_bg.data;
             sex_df.data = sex_bg.data;
     }else if(cb_obj.value === "数据挖掘组"){
             dep_df.data = dep_dm.data;
             sbj_df.data = sbj_dm.data;
             sex_df.data = sex_dm.data;
     }else if(cb_obj.value === "前端组"){
             dep_df.data = dep_web.data;
             sbj_df.data = sbj_web.data;
             sex_df.data = sex_dm.data;
     }else if(cb_obj.value === "嵌入式组"){
             dep_df.data = dep_eb.data;
             sbj_df.data = sbj_eb.data;
             sex_df.data = sex_dm.data;
     }else if(cb_obj.value === "移动组"){
             dep_df.data = dep_app.data;
             sbj_df.data = sbj_app.data;
             sex_df.data = sex_app.data;
     }else if(cb_obj.value === "图形组"){
             dep_df.data = dep_gf.data;
             sbj_df.data = sbj_gf.data;
             sex_df.data = sex_app.data;
     }else if(cb_obj.value === "设计组"){
             dep_df.data = dep_ds.data;
             sbj_df.data = sbj_ds.data;
             sex_df.data = sex_ds.data;
     }
     dep_df.change.emit();
     sbj_df.change.emit();
     sex_df.change.emit();
     """)

select.js_on_change('value', callback)




# 分布图
mu, sigma = 3.3, 0.3
group_list = list()
for group in groups[::-1]:
    if group == '总体':
        measured = stu_df.point.values
    else:
        measured = stu_df[stu_df['group'] == group].point.values
    hist, edges = np.histogram(measured, density=True, bins=50)
    hist_src = ColumnDataSource(data=dict(hist=hist))

    x = np.linspace(2, 5, 50)
    pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))

    p = figure(title=group+"绩点分布", tools='', background_fill_color="#fafafa")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="navy", line_color="white", alpha=0.5)
    p.line(x, pdf, line_color="#ff8888", line_width=4, alpha=0.7, legend_label='正态分布')

    p.y_range.start = 0
    p.xaxis.axis_label = '绩点'
    p.yaxis.axis_label = '概率密度'
    p.grid.grid_line_color="white"
    group_list.append(p)

layout_p = layout([
    [p_num],
    [column(select, row(p_dpm, p_spj))],
    [row(p_sex, p_qg)],
    column(group_list)
], height_policy='fit')

show(layout_p)

In [6]:
dict(stu_df.department.value_counts())

{'信息安全2班': 10,
 '计算机科学与技术6班': 9,
 '网络工程2班': 9,
 '软件工程5班': 9,
 '软件工程3班': 9,
 '计算机科学与技术5班': 8,
 '计算机科学与技术1班': 7,
 '软件工程2班': 6,
 '网络工程4班': 6,
 '计算机科学与技术2班': 6,
 '软件工程1班': 6,
 '信息安全1班': 6,
 '网络工程1班': 6,
 '网络工程3班': 5,
 '计算机科学与技术': 5,
 '电子信息类10班': 5,
 '数据科学与大数据技术2班': 4,
 '电子信息类7班': 4,
 '计算机科学与技术4班': 4,
 '软件工程一班': 3,
 '电子信息类8班': 3,
 '计算机科学与技术3班': 3,
 '电子信息类13班': 3,
 '电子信息类4班': 3,
 '信息安全一班': 3,
 '软件工程4班': 3,
 '光电信息科学与工程3班': 2,
 '电子信息类5班': 2,
 '自动化5班': 2,
 '机械类7班': 2,
 '电子信息类6班': 2,
 '电子科学与技术4班': 2,
 '电子科学与技术1班': 2,
 '电子信息类三班': 2,
 '电子信息类12班': 2,
 '软件工程二班': 2,
 '计科6班': 2,
 '食品科学与工程': 1,
 '能源与动力工程3班': 1,
 '数据科学与大数据技术专业2班': 1,
 '物联网工程1班': 1,
 '信息安全': 1,
 '安全工程1班': 1,
 '电子科学与技术（5）班': 1,
 '软件工程3': 1,
 '应用化学3班': 1,
 '工程管理1班': 1,
 '电子信息类3班': 1,
 '电子信息类7班 ': 1,
 '测控技术与仪器1班': 1,
 '电子信息类': 1,
 '计算机科学与技术四班': 1,
 '计算机科学与技术六班': 1,
 '物联网工程2班': 1,
 '生物工程2班': 1,
 '数据科学与大数据技术1班': 1,
 '法学1班': 1,
 '材料成型及控制工程3班': 1,
 '电子信息类（7）班': 1,
 '土木': 1,
 '电子科学与技术': 1,
 '电子信息类9班': 1,
 '计科四班': 1,
 '测控技术与仪器2班': 1,
 '计科3班': 1,
